In [2]:
pip install backtrader

     ---------------------------------------- 0.0/419.5 kB ? eta -:--:--
      --------------------------------------- 10.2/419.5 kB ? eta -:--:--
     -- ---------------------------------- 30.7/419.5 kB 435.7 kB/s eta 0:00:01
     -- ---------------------------------- 30.7/419.5 kB 435.7 kB/s eta 0:00:01
     ------ ------------------------------ 71.7/419.5 kB 393.8 kB/s eta 0:00:01
     --------- -------------------------- 112.6/419.5 kB 504.4 kB/s eta 0:00:01
     ------------- ---------------------- 153.6/419.5 kB 573.4 kB/s eta 0:00:01
     ---------------- ------------------- 194.6/419.5 kB 655.9 kB/s eta 0:00:01
     -------------------- --------------- 235.5/419.5 kB 656.8 kB/s eta 0:00:01
     ----------------------- ------------ 276.5/419.5 kB 741.6 kB/s eta 0:00:01
     --------------------------- -------- 317.4/419.5 kB 728.0 kB/s eta 0:00:01
     ------------------------------- ---- 368.6/419.5 kB 740.5 kB/s eta 0:00:01
     -----------------------------------  409.6/419.5


[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install yfinance

     ---------------------------------------- 0.0/60.3 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.3 kB 660.6 kB/s eta 0:00:01
     -------------------------------- ----- 51.2/60.3 kB 525.1 kB/s eta 0:00:01
     -------------------------------------- 60.3/60.3 kB 538.6 kB/s eta 0:00:00
     ---------------------------------------- 0.0/62.5 kB ? eta -:--:--
     ------ --------------------------------- 10.2/62.5 kB ? eta -:--:--
     ---------------------------------------- 62.5/62.5 kB 1.1 MB/s eta 0:00:00
  Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
     ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.8 MB 660.6 kB/s eta 0:00:06
      --------------------------------------- 0.1/3.8 MB 787.7 kB/s eta 0:00:05
     - -------------------------------------- 0.1/3.8 MB 939.4 kB/s eta 0:00:04
     - -------------------------------------- 0.2/3.8 MB 919.0 kB/s eta 0:00:04
     - ----


[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import backtrader as bt
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell


In [5]:
InteractiveShell.ast_node_interactivity = "all"

In [6]:
#downloading the data
data = yf.download('TSLA',start='2019-01-01',end='2021-12-31')

[*********************100%***********************]  1 of 1 completed


In [7]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,20.406668,21.008667,19.920000,20.674667,20.674667,174879000
2019-01-03,20.466667,20.626667,19.825333,20.024000,20.024000,104478000
2019-01-04,20.400000,21.200001,20.181999,21.179333,21.179333,110911500
2019-01-07,21.448000,22.449333,21.183332,22.330667,22.330667,113268000
2019-01-08,22.797333,22.934000,21.801332,22.356667,22.356667,105127500
...,...,...,...,...,...,...
2021-12-23,335.600006,357.660004,332.519989,355.666656,355.666656,92713200
2021-12-27,357.890015,372.333344,356.906677,364.646667,364.646667,71145900
2021-12-28,369.829987,373.000000,359.473328,362.823334,362.823334,60324000


In [8]:
#converting the data to backtrader feeds
data = bt.feeds.PandasData(dataname=data)

In [9]:
type(data)

backtrader.feeds.pandafeed.PandasData

In [10]:
cerebro = bt.Cerebro()

In [11]:
cerebro.adddata(data)

In [12]:
#creating a strategy class
class TradeStrategy(bt.Strategy):
  #defining indicators to be used
  def __init__(self):
    self.sma = bt.indicators.SimpleMovingAverage(period=50)
    self.ema = bt.indicators.ExponentialMovingAverage(period=14)
    self.rsi = bt.indicators.RelativeStrengthIndex()
  #creating strategy 
  def next(self):
    if self.ema>self.sma:
      self.buy(size=30)
    elif self.ema<self.sma:
      self.sell(size=30)
    else:
      pass


In [13]:
#adding strategy to the cerebro
cerebro.addstrategy(TradeStrategy)

0

In [14]:
#adding multiple analyzer to the cerebro
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='annualreturn')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')

In [15]:
#setting initial cash 
cerebro.broker.setcash(100000.0)

In [16]:
#printing the results

print(f"Start:{cerebro.broker.get_value()}")
print(f"StartCash:{cerebro.broker.get_cash()}")
thestrats = cerebro.run()
thestrat = thestrats[0]
cerebro.run() #operating cerebro to run the strategy
print("")
print(f"End:{cerebro.broker.get_value()}")
print(f"EndCash:{cerebro.broker.get_cash()}")
print(f"SharpeRatio:{thestrat.analyzers.mysharpe.get_analysis()}")
print(f"AnnualReturn:{thestrat.analyzers.annualreturn.get_analysis()}")
print(f"Drawdown:{thestrat.analyzers.drawdown.get_analysis()}")
print(f"Returns:{thestrat.analyzers.returns.get_analysis()}")

Start:100000.0
StartCash:100000.0



End:572908.8366889954
EndCash:91255.83833694458
SharpeRatio:OrderedDict([('sharperatio', 0.7870028548475487)])
AnnualReturn:OrderedDict([(2019, -0.09168940143585202), (2020, 3.5489739923713763), (2021, 0.3865568647736002)])
Drawdown:AutoOrderedDict([('len', 38), ('drawdown', 13.82599889606377), ('moneydown', 91919.10369873047), ('max', AutoOrderedDict([('len', 188), ('drawdown', 48.02902182606897), ('moneydown', 178274.30786132812)]))])
Returns:OrderedDict([('rtot', 1.7455564197951707), ('ravg', 0.0023089370632211254), ('rnorm', 0.7893494891724915), ('rnorm100', 78.93494891724914)])


In [25]:
#plotting the trades 
%matplotlib inline
plt.rcParams['figure.figsize'] = [15, 18]
plt.rcParams.update({'font.size': 12}) 
cerebro.plot()

<IPython.core.display.Javascript object>

[[<Figure size 1500x1800 with 5 Axes>]]